In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# 데이터 확인
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Dataset 만들기
import keras
from tensorflow.keras.utils import to_categorical

# Detect Face
import cv2
from scipy.ndimage import zoom

# Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Importing necessary layers and modules from TensorFlow instead of standalone Keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 전체 이미지에서 얼굴을 찾아내는 함수
def detect_face(frame):

    # cascade pre-trained 모델 불러오기
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # RGB를 gray scale로 바꾸기
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # cascade 멀티스케일 분류
    detected_faces = face_cascade.detectMultiScale(gray,
                                                   scaleFactor = 1.1,
                                                   minNeighbors = 6,
                                                   minSize = (shape_x, shape_y),
                                                   flags = cv2.CASCADE_SCALE_IMAGE
                                                  )

    coord = []
    for x, y, w, h in detected_faces:
        if w > 100:
            sub_img = frame[y:y+h, x:x+w]
            coord.append([x, y, w, h])

    return gray, detected_faces, coord

# 전체 이미지에서 찾아낸 얼굴을 추출하는 함수
def extract_face_features(gray, detected_faces, coord, offset_coefficients=(0.075, 0.05)):
    new_face = []
    for det in detected_faces:

        # 얼굴로 감지된 영역
        x, y, w, h = det

        # 이미지 경계값 받기
        horizontal_offset = int(np.floor(offset_coefficients[0] * w))
        vertical_offset = int(np.floor(offset_coefficients[1] * h))

        # gray scacle 에서 해당 위치 가져오기
        extracted_face = gray[y+vertical_offset:y+h, x+horizontal_offset:x-horizontal_offset+w]

        # 얼굴 이미지만 확대
        new_extracted_face = zoom(extracted_face, (shape_x/extracted_face.shape[0], shape_y/extracted_face.shape[1]))
        new_extracted_face = new_extracted_face.astype(np.float32)
        new_extracted_face /= float(new_extracted_face.max()) # sacled
        new_face.append(new_extracted_face)

    return new_face

shape_x = 96
shape_y = 96

from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from PIL import Image
import io

# 모델 로드
model_path = '/content/drive/MyDrive/Model/model.keras'  # Update the path as needed
model = load_model(model_path)
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

def take_photo(filename, quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = 'Capture';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            // Resize the output to fit the video element.
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            // Wait for Capture to be clicked.
            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

emotion_labels = ['angry', 'disgust', 'happy', 'sad', 'neutral']  # Ensure this aligns with the model's output

In [ ]:
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

def analyze_emotion(image_path, model):
    face = cv2.imread(image_path)

    gray, detected_faces, coord = detect_face(face)
    if len(detected_faces) == 0:
        print("No face detected in the image.")
        return
    face_zoom = extract_face_features(gray, detected_faces, coord)

    img_array = np.reshape(face_zoom[0], (1, 96, 96, 1))

    prediction = model.predict(img_array)[0] 

    plt.subplot(121)
    plt.title("Original Face")
    plt.imshow(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))

    plt.subplot(122)
    plt.title("Extracted Face")
    plt.imshow(face_zoom[0], cmap='gray')
    plt.axis('off')
    plt.show()

    return prediction


from IPython.display import display, HTML
import ipywidgets as widgets

model_1 = widgets.Checkbox(value=False, description='Model 1')
model_2 = widgets.Checkbox(value=False, description='Model 2')
model_3 = widgets.Checkbox(value=False, description='Model 3')

submit_button = widgets.Button(description="Submit")

output = widgets.Output()

selected_models_global = []

def on_submit_click(b):
    global selected_models_global 
    with output:
        output.clear_output()
        selected_models_global = []

        if model_1.value:
            selected_models_global.append(1)
            model_path = '/content/drive/MyDrive/Model/sad.keras' 
            model = load_model(model_path)
        if model_2.value:
            selected_models_global.append(2)
            model_path = '/content/drive/MyDrive/Model/angry.keras' 
            model = load_model(model_path)
        if model_3.value:
            selected_models_global.append(3)
            model_path = '/content/drive/MyDrive/Model/happy.keras' 
            model = load_model(model_path)

        if selected_models_global:
            print(f"You selected model: {', '.join(map(str, selected_models_global))}")
        else:
            print("No model selected!")

submit_button.on_click(on_submit_click)

display(HTML("<h1>Select Model to Use</h1>"))
display(model_1, model_2, model_3, submit_button, output)

In [ ]:
import numpy as np

def calculate_variance(predictions):
    """
    Calculate the variance for the given predictions.
    Each prediction is expected to be a list or array of emotion probabilities.
    """
    if len(predictions) < 2:
        print("Not enough data to calculate variance.")
        return None

    predictions_array = np.array(predictions)  
    variance = np.var(predictions_array, axis=0) 
    return variance



while selected_models_global:
      try:

          print("Show your happy face! Do not exaggerate and just show your natural happy face.")
          happy = take_photo('happy.jpg')
          prediction_happy = analyze_emotion(happy, model)
          print("Show your sad face! Do not exaggerate and just show your natural sad face.")
          sad = take_photo('sad.jpg')
          prediction_sad = analyze_emotion(sad, model)
          print("Show your angry face! Do not exaggerate and just show your natural angry face.")
          angry = take_photo('angry.jpg')
          prediction_angry = analyze_emotion(angry, model)
          print("Make a facial expression that you usually make in your daily life!")
          usual = take_photo('usual.jpg')
          prediction_usual = analyze_emotion(usual, model)


          print("Photo taken! Analyzing your expression type...")

          predictions = [prediction_happy, prediction_sad, prediction_angry, prediction_usual]

          variance = calculate_variance(predictions)

          total_variance_sum = sum(variance)

          if selected_models_global[0] == 1:
            if total_variance_sum > 3.3205783900314145e-02:
              final_type = 'SAOP(Sad+Opened)' 
            else:
              final_type = 'SACL(Sad+Closed)'
          elif selected_models_global[0] == 2:
            if total_variance_sum > 2.7631465258082244e-01:
              final_type = 'ANOP(Angry+Opened)' 
            else:
              final_type = 'ANCL(Angry+Closed)'
          else:
            if total_variance_sum > 9.196208488049251e-02:
              final_type = 'HAOP(Happy+Opened)' 
            else:
              final_type = 'HACL(Happy+Closed)'


          print("Finished analyzing!")
          print("Your e:MBTI is...", final_type)
          break  # 성공적으로 실행되었으므로 루프 종료

      except Exception as e:
            print("An error occurred:", str(e))
            print("Retrying...")